In [1]:
import torch
import os
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

# 1.加载并标准化CIFAR10

In [2]:
# author:banbao
# version:2
# Maxpooling -> FractionalMaxPool2d
# dropout

# ToTensor 可以把一个PIL或numpy图片转为torch.Tensor
# Normaize 正则化(预处理)
# torchvision.transforms.Normalize(mean, std, inplace=False)
# input[channel] = (input[channel] - mean[channel]) / std[channel]
# 如下示例,输入原本为 [0,1] -> [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 训练集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# 测试集
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# 类别
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

# 2.定义卷积神经网络

In [4]:
# VGG16
# 原始:CCP,CCP,CCCP,CCCP,CCCP,softmax
# 卷积核大小都为 3*3,步长为 1, padding=1(保持图片大小不变)
# N=4,每组 4 个样本
# 每个样本过程如下
# 1. 3*32*32    ->  卷积核个数:64
# 2. 64*32*32   ->  卷积核个数:64
# 3. 64*32*32   ->  maxpool:2*2
# 4. 64*16*16   ->  卷积核个数:128
# 5. 128*16*16  ->  卷积核个数:128
# 6. 128*16*16  ->  maxpool:2*2
# 7. 128*8*8    ->  卷积核个数:256
# 8. 256*8*8    ->  卷积核个数:256
# 9. 256*8*8    ->  卷积核个数:256
#10. 256*8*8    ->  maxpool:2*2
#11. 256*4*4    ->  卷积核个数:512
#12. 512*4*4    ->  卷积核个数:512
#13. 512*4*4    ->  卷积核个数:512
#14. 512*4*4    ->  maxpool:2*2
#15. 512*2*2    ->  卷积核个数:512
#16. 512*2*2    ->  卷积核个数:512
#17. 512*2*2    ->  卷积核个数:512
#18. 512*2*2    ->  maxpool:2*2
#19. 512*1*1    ->  Linear:512->10
#20. 1

# torch.nn.Conv2d(in_channels, out_channels, kernel_size, 
# stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

# torch.nn.AvgPool2d(kernel_size, stride=None, padding=0, 
# ceil_mode=False, count_include_pad=True, divisor_override=None)

In [5]:
# 定义卷积神经网络
class Net(nn.Module):
    
    # 构造函数
    def __init__(self):
        super(Net, self).__init__()
        # 网络层
        self.layers = nn.Sequential(
            # CCP
            nn.Conv2d(3,  64, 3, stride=1, padding=1),
            # 正则化
            nn.BatchNorm2d(64),
            # 激活函数
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.FractionalMaxPool2d(kernel_size=2, output_ratio=(0.5, 0.5)),
            # CCP
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128,128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.FractionalMaxPool2d(kernel_size=2, output_ratio=(0.5, 0.5)),
            # CCCP
            nn.Conv2d(128,256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.FractionalMaxPool2d(kernel_size=2, output_ratio=(0.5, 0.5)),
            # CCCP
            nn.Conv2d(256,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.FractionalMaxPool2d(kernel_size=2, output_ratio=(0.5, 0.5)),
            # CCCP
            # dropout-v3
            nn.Dropout(p=0.2),
            # dropout-v3
            nn.Conv2d(512,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.FractionalMaxPool2d(kernel_size=2, output_ratio=(0.5, 0.5)),
            # dropout-v3
            nn.Dropout(p=0.2)
            # dropout-v3
        )
        # Linear
        self.classifier = nn.Linear(512, 10)
    
    # 网络层的构建
    def forward(self, x):
        output = self.layers(x)
        # 一个 batch 有4个样本
        output = output.view(4,512)
        output = self.classifier(output)
        return output

In [6]:
net = Net()
print(net)

Net(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, 

# 3.定义损失函数和优化器

In [7]:
# 交叉熵
criterion = nn.CrossEntropyLoss()
# 随机梯度下降(momentum)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 4.训练网络

## GPU

In [8]:
useGPU = torch.cuda.is_available()

## 4.2 测试函数(计算ACC)

In [9]:
def calcAcc(isTrain):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in trainloader if isTrain else testloader:
            images, labels = data
            if useGPU:
                images = images.cuda()
                labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 1.0*correct/total

## 4.3 不同类型的准确率

In [10]:
def showSpecificClass():
    # 看看不同类别的准确率
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            if useGPU:
                images = images.cuda()
                labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

## 4.4 训练网络

In [11]:
now = []
def train(start, end):
    # 多次迭代
    for epoch in range(start, end):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # 输入
            inputs, labels = data
            if useGPU:
                inputs = inputs.cuda()
                labels = labels.cuda()
            # 梯度清0
            optimizer.zero_grad()
            # 训练
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # 损失函数
            running_loss += loss.item()
            # 每间隔 500*4 样本输出一个示意符号
            if i % 500 == 499:
                print("#",end="")
        # 输出当前的正确率
        trainAcc = calcAcc(True)
        testAcc = calcAcc(False)
        now = [trainAcc, testAcc, epoch]
        print("\n[{}],loss: {}, train acc: {}, test acc: {}".format(epoch, running_loss, trainAcc, testAcc))
        # if((epoch % 10 == 0) or (epoch == end-1)):
        if not os.path.exists("./model"):
            os.makedirs('./model')
        PATH = "./model/GPU-model-VGG-03-trainacc-{}-testacc-{}-epoch-{}.pkl".format(now[0], now[1], now[2])
        torch.save(net.state_dict(), PATH)
    print('Finished Training')

## 4.5 运行

In [12]:
# GPU
if useGPU:
    net = net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    torch.backends.cudnn.benchmark = True
    criterion.cuda()
    # optimizer.cuda()
    print("USING GPU(CUDA)!")
else:
    print("USING CPU!")

USING GPU(CUDA)!


In [13]:
# 训练
# train(0,2)
# train(0,100)

In [14]:
# 加载已经保存的模型
# PATH = "./model/GPU-model-VGG-03-trainacc-0.98556-testacc-0.8247-epoch-19.pkl"
# net.load_state_dict(torch.load(PATH))